# ***Desafio 2***

O objetivo do desafio é construir um algorítmo de sorting que dado o dataset e as restrições, nos devolva a listagem final ordenando os ítens:

1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla(más fuerte que las demás.)
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla(más fuerte que las demás.)
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECO”(siendo esta regla más fuerte que la 1 y 2.)
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de mayor score a menor.


In [1]:
import pandas as pd
import time

Vamos ler o arquivo de dados

In [4]:
def ler_df(nome):
    df = pd.read_csv(nome) 
    print(df.info())
    return df

In [5]:
nome_arquivo = 'ordenamiento.csv'
df = ler_df(nome_arquivo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item_id   5000 non-null   int64  
 1   vertical  5000 non-null   object 
 2   category  5000 non-null   object 
 3   domain    5000 non-null   object 
 4   score     5000 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 195.4+ KB
None


Pecebemos que o dataframe não possui valores nulos

Verifico se os itens destacados nas regras 3 e 4 existem

In [95]:
df[df['item_id']==641416750].head()

,item_id,vertical,category,domain,score
1617,641416750,CE,MOBILE,MLC-CELLPHONES,0.9986


In [96]:
df[df['item_id']==22351223].head()

,item_id,vertical,category,domain,score


Ordeno o dataframe:

In [6]:
df_ordenado = df.sort_values('score', ascending = False).reset_index(drop=True)
df.head(10)

,item_id,vertical,category,domain,score
0,512208310,CPG,PETS FOOD,MLC-CATS_AND_DOGS_FOODS,0.0272
1,468513076,CE,ELECTRONICS,MLC-RANGES,0.9256
2,614337410,CE,ELECTRONICS,MLC-SMART_SPEAKERS,0.8304
3,634351318,APP & SPORTS,APPAREL,MLC-PANTS,0.0560
4,528383704,ACC,VEHICULAR MULTIMEDIA,MLC-GPS,0.2334
5,507642148,HOME & INDUSTRY,HOME&DECOR,MLC-STOOLS,0.3796
6,521057099,HOME & INDUSTRY,HOME&DECOR,MLC-QUILTS_AND_COVERLETS,0.4352
7,529123507,CE,ELECTRONICS,MLC-BLENDERS,0.2476
8,630745490,HOME & INDUSTRY,HOME&DECOR,MLC-CORKSCREWS,0.6292
9,627255896,ACC,AUTOPARTS,MLC-VEHICLE_MULTIMEDIA_SYSTEMS,0.1804


Criamos agora a função de ordenamendo:

In [8]:
def sorting_meli(data):

    start = time.time()
    data = data.sort_values('score', ascending = False).reset_index(drop=True)

# Aplico a condição 2 nos primeiros 9 registros, pois a condição 1 não pode acontecer devido às regras mais fortes, 
# faremos o tratamento da condição 1 depois da condição 5
    for idx in range(0,9):
        data_temp = data.iloc[idx: , :]
        data_temp = data_temp[data_temp["vertical"] != data_temp["vertical"][idx]]
        idx_new_value = data_temp.index.values[0]
        data.loc[idx + 0.5] = data.iloc[idx_new_value]
        data.drop(data.index[idx_new_value], inplace = True)
        data = data.sort_index().reset_index(drop=True)
        
# Condição 3, procuro se existe o ID:641416750 no dataset original ordenado, pego o registro e insiro na posição 4 do dataframe
    if data["item_id"].isin([641416750]).any():
        idx = data[data["item_id"].isin([641416750])].index.values[0]
        data.loc[1.5] = data.iloc[idx]
        data = data.drop(data.index[idx])
        data = data.sort_index().reset_index(drop=True)
        
# Condição 4, procuro se existe o ID:22351223 no dataset original ordenado, pego o registro e insiro na posição 6 do dataframe       
    if data["item_id"].isin([22351223]).any():
        idx = data[data["item_id"].isin([22351223])].index.values[0]
        data.loc[4.5] = data.iloc[idx]
        data = data.drop(data.index[idx])
        data = data.sort_index().reset_index(drop=True)

#Condição 5, procuro no dataframe ordenado os 3 primeiros registros de "HOME&DECOR" e insiro nas posições 9,10,11
    for idx in range(8, 11):
        data_temp = data.iloc[idx: , :]
        data_temp = data_temp[data_temp["category"] == "HOME&DECOR"]
        idx_new_value = data_temp.index.values[0]
        data.loc[idx - 0.5] = data_temp.iloc[idx_new_value]
        data.drop(data.index[idx_new_value], inplace = True)
        data = data.sort_index().reset_index(drop=True)
        
#Agora vamos aplicar a condição 1 para o resto do dataframe
    count = 0
    for idx in range(10, len(data)):
        data_temp = data.iloc[idx: , :]
        
        try:
            if data_temp["domain"][idx] == data_temp["domain"][idx + 1]:
                count += 1                
                if count == 4:
                    data_temp = data_temp[data_temp["domain"] != data_temp["domain"][idx]]
                    idx_new_value = data_temp.index.values[0]
                    data.loc[idx + 0.5] = data.iloc[idx_new_value]
                    data.drop(data.index[idx_new_value], inplace = True)
                    data = data.sort_index().reset_index(drop=True)
            else:
                count = 0 
        except:
            break
    
#Agora vamos aplicar a condição 2 para o resto do dataframe
    for idx in range(10, len(data)):
        data_temp = data.iloc[idx: , :]
        data_temp = data_temp[data_temp["vertical"] != data_temp["vertical"][idx]]
        try:
            idx_new_value = data_temp.index.values[0]
            data.loc[idx + 0.5] = data.iloc[idx_new_value]
            data.drop(data.index[idx_new_value], inplace = True)
            data = data.sort_index().reset_index(drop=True)
        except:
            break       
        
        
    
    end = time.time()
    T = end - start
    print("The seconds used to execute this program is: ", T)
    
    return data

In [9]:
df_ordenado_meli = sorting_meli(df)
df_ordenado_meli.head(30)

The seconds used to execute this program is:  22.49313974380493


,item_id,vertical,category,domain,score
0,590602034,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
1,523534468,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9976
2,641416750,CE,MOBILE,MLC-CELLPHONES,0.9986
3,609438042,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9996
4,541283090,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-ELECTRIC_DRILLS,0.9968
5,634352041,CE,MOBILE,MLC-CELLPHONES,0.9994
6,610865341,HOME & INDUSTRY,INDUSTRY,MLC-POINTS_OF_SALE_KITS,0.9960
7,615879515,CE,MOBILE,MLC-CELLPHONES,0.9992
8,915073939,HOME & INDUSTRY,HOME&DECOR,MLC-MATTRESSES,0.8130
9,569077539,HOME & INDUSTRY,HOME&DECOR,MLC-SUNBATHING_CHAIRS,0.7938
